# Creating SDSS dataset from SDSS DR15 Query

    --> http://skyserver.sdss.org/dr15/en/tools/search/sql.aspx

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import warnings
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, normalize 
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import timeit
import os
warnings.filterwarnings("ignore")

In [10]:
all_filenames = ['new input/QSO.csv','new input/Galaxy.csv','new input/Star.csv']
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "SDSS_combined.csv", index=False, encoding='utf-8-sig')

In [11]:
df = pd.read_csv('SDSS_combined.csv')

In [12]:
df = df.sample(frac = 1)
df.head()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
10682,1.237650e+18,210.271857,-1.141683,18.93601,17.53573,16.95067,16.59856,16.38827,752,301,1,446,3.389090e+17,GALAXY,0.073794,301,51942,48
28374,1.237649e+18,182.601147,-1.003993,19.31073,17.73028,17.15447,16.92641,16.80567,756,301,1,446,3.637234e+17,STAR,0.000128,323,51615,210
6557,1.237650e+18,347.457406,13.752903,19.36419,19.32540,19.18805,18.90424,18.71285,1739,301,2,221,6.920040e+18,QSO,1.982645,6146,56265,930
10984,1.237650e+18,226.286496,0.118770,18.75197,17.30061,16.56847,16.15376,15.85280,752,301,4,553,3.491690e+17,GALAXY,0.070359,310,51990,508
4463,1.237650e+18,199.913856,2.708801,19.44041,19.25368,19.30330,19.19903,19.27240,1458,301,4,501,5.923400e+17,QSO,1.676901,526,52312,426


In [13]:
sdss_df = df
sdss_df

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
10682,1.237650e+18,210.271857,-1.141683,18.93601,17.53573,16.95067,16.59856,16.38827,752,301,1,446,3.389090e+17,GALAXY,0.073794,301,51942,48
28374,1.237649e+18,182.601147,-1.003993,19.31073,17.73028,17.15447,16.92641,16.80567,756,301,1,446,3.637234e+17,STAR,0.000128,323,51615,210
6557,1.237650e+18,347.457406,13.752903,19.36419,19.32540,19.18805,18.90424,18.71285,1739,301,2,221,6.920040e+18,QSO,1.982645,6146,56265,930
10984,1.237650e+18,226.286496,0.118770,18.75197,17.30061,16.56847,16.15376,15.85280,752,301,4,553,3.491690e+17,GALAXY,0.070359,310,51990,508
4463,1.237650e+18,199.913856,2.708801,19.44041,19.25368,19.30330,19.19903,19.27240,1458,301,4,501,5.923400e+17,QSO,1.676901,526,52312,426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19088,1.237650e+18,171.518387,0.748348,19.52704,18.21794,17.49654,17.18454,17.02325,756,301,5,372,3.154120e+17,GALAXY,0.180377,280,51612,583
7615,1.237650e+18,2.875770,-10.398289,19.38607,19.23572,18.92413,18.81626,18.32608,1729,301,3,102,7.341730e+17,QSO,0.337151,652,52138,314
25293,1.237649e+18,139.937188,-0.032528,17.05420,15.76948,15.74572,15.77202,15.77638,756,301,3,161,5.337750e+17,STAR,0.000182,474,52000,358
5422,1.237650e+18,195.372712,2.646836,19.58305,19.27771,19.20473,18.93744,18.86707,1462,301,4,467,5.890210e+17,QSO,1.862723,523,52026,637


In [14]:
#Handling missing values
df = df.dropna()
sdss_df_fe = df

# Preprocess Functions

In [15]:
sdss_df_fe = sdss_df_fe.sample(frac=1)

In [16]:
sdss_df_fe

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
2126,1.237650e+18,185.182780,1.189488,19.34126,18.46700,18.44940,18.33074,18.06268,756,301,6,463,4.330450e+18,QSO,2.627589,3846,55327,865
11556,1.237650e+18,216.927113,0.824043,19.20285,18.19531,17.59929,17.22163,17.02410,756,301,5,675,3.446170e+17,GALAXY,0.150541,306,51637,335
28889,1.237651e+18,130.183176,49.955453,18.53200,17.65095,17.24701,17.12667,17.10159,1331,301,2,175,8.231498e+18,STAR,-0.000139,7311,57038,158
8502,1.237650e+18,146.011531,59.079110,18.68013,18.36549,18.32169,18.33179,18.27370,1331,301,4,261,5.100870e+17,QSO,0.775121,453,51915,196
8144,1.237660e+18,190.489753,63.544891,19.15626,18.12416,17.64571,17.40442,17.13662,2304,301,6,214,8.794520e+17,QSO,2.613389,781,52373,450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28203,1.237652e+18,245.382180,51.272421,19.34043,18.26579,17.90533,17.77829,17.70416,1412,301,5,389,3.702108e+18,STAR,-0.000553,3288,54908,542
28357,1.237649e+18,181.406660,-1.017241,17.77664,16.16791,15.59226,15.40551,15.33608,756,301,1,438,3.256127e+18,STAR,0.000045,2892,54552,87
24456,1.237650e+18,8.672036,15.573279,19.57385,18.31827,17.81188,17.61035,17.51804,1035,301,5,21,3.527563e+18,STAR,-0.000095,3133,54789,431
27335,1.237651e+18,112.956179,37.318673,17.47377,16.36555,16.08178,15.97640,15.96756,1402,301,4,32,2.311624e+18,STAR,-0.000028,2053,53446,551


In [30]:
sdss_df_fe = sdss_df

# encode class labels to integers
le = LabelEncoder()
y_encoded = le.fit_transform(sdss_df_fe['class'])
sdss_df_fe['class'] = y_encoded



In [17]:
sdss_df_fe.to_csv('input/SDSS_dr16.csv',index=False)

In [18]:
os.remove("SDSS_combined.csv")